In [17]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib as plt

import torch
import implicit

import sys
sys.path.append('..')

from src.datasets import load_dataset
from src.evaluation.evaluation import downvote_seen_items, topn_recommendations, model_evaluate
from src.models import NGCF, ALS, eALS, iALS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
topn = 10


def evaluate_model(model, holdout, testset, data_description, topn=10, verbose=False):
    scores = model.score_users(holdout.userid.values)
    downvote_seen_items(scores, testset, data_description)
    recs = topn_recommendations(scores, topn=topn)

    hr, mrr, ndcg, cov = model_evaluate(recs, holdout, data_description, topn=topn)

    if verbose:
        print(f'HR@{topn} = {hr:.4f}')
        print(f'MRR@{topn} = {mrr:.4f}')
        print(f'nDCG@{topn} = {ndcg:.4f}')
        print(f'COV@{topn} = {cov:.4f}')

    return hr, mrr, ndcg, cov


def score_models(models, holdout, testset, data_description):
    scores = dict()
    for name, model in models:
        scores[name] = evaluate_model(model, holdout, testset, data_description)

    scores = pd.DataFrame(scores, index=['HR', 'MRR', 'NDCG', 'COV']).T
    return scores


def evaluate_implicit_model(implicit_model, holdout, training_matrix, data_description, topn=10, verbose=False):
    model = implicit_model(iterations=100, regularization=0.1)
    model.fit(training_matrix)

    recs = model.recommend_all(training_matrix, filter_already_liked_items=True, N=topn)
    hr, mrr, ndcg, cov = model_evaluate(recs, holdout, data_description, topn=topn)

    if verbose:
        print(f'HR@{topn} = {hr:.4f}')
        print(f'MRR@{topn} = {mrr:.4f}')
        print(f'nDCG@{topn} = {ndcg:.4f}')
        print(f'COV@{topn} = {cov:.4f}')

    return hr, mrr, ndcg, cov

### Load Data

In [3]:
data_path = '../data'

training_yelp, testset_yelp, holdout_yelp, training_matrix_yelp, data_description_yelp = load_dataset('yelp', data_path)
training_movielens, testset_movielens, holdout_movielens, training_matrix_movielens, data_description_movielens = load_dataset('movielens', data_path)

### Load Pretrained Models

In [27]:
models_movielens = [
    ('ALS', ALS.from_checkpoint('../weights/als_movielens.pkl')),
    ('eALS', eALS.from_checkpoint('../weights/eals_movielens.pkl')),
    ('iALS', iALS.from_checkpoint('../weights/ials_movielens.pkl')),
    ('NGCF', torch.load('../weights/ngcf_movielens.pth'))
]

models_yelp = [
    ('ALS', ALS.from_checkpoint('../weights/als_yelp.pkl')),
    ('eALS', eALS.from_checkpoint('../weights/eals_yelp.pkl')),
    ('iALS', iALS.from_checkpoint('../weights/ials_yelp.pkl')),
    ('NGCF', torch.load('../weights/ngcf_yelp.pth'))
]

### Score Models

In [57]:
scores_movielens = score_models(models_movielens, holdout_movielens, testset_movielens, data_description_movielens)
scores_movielens.style.background_gradient()

,HR,MRR,NDCG,COV
ALS,0.102318,0.036046,0.051282,0.367171
eALS,0.099338,0.035760,0.050432,0.364201
iALS,0.101325,0.035419,0.050591,0.366091
NGCF,0.082450,0.026750,0.039567,0.674946


In [32]:
scores_yelp = score_models(models_yelp, holdout_yelp, testset_yelp, data_description_yelp)
scores_yelp.style.background_gradient()

,HR,MRR,NDCG,COV
ALS,0.041477,0.015488,0.021485,0.062410
eALS,0.041048,0.015388,0.021323,0.061712
iALS,0.040854,0.015152,0.021101,0.061906
NGCF,0.037154,0.013125,0.018649,0.143373


### Comparison with Implicit

In [52]:
def score_implicit_models(implicit_models, holdout, training_matrix, data_description):
    scores = dict()
    for name, model in implicit_models:
        metrics = evaluate_implicit_model(model, holdout, training_matrix, data_description, topn=topn)
        scores[name] = metrics

    scores = pd.DataFrame(scores, index=['HR', 'MRR', 'NDCG', 'COV']).T
    return scores

In [51]:
implicit_models = [
    ('ALS-impl', implicit.als.AlternatingLeastSquares), 
    ('BPR-impl', implicit.bpr.BayesianPersonalizedRanking)
]

scores_implicit_movielens = score_implicit_models(implicit_models, holdout_movielens, training_matrix_movielens, data_description_movielens)
scores_implicit_movielens.style.background_gradient()

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

,HR,MRR,NDCG,COV
ALS-impl,0.097682,0.034040,0.048701,0.414147
BPR-impl,0.053974,0.019872,0.027741,0.250270


In [53]:
scores_implicit_yelp = score_implicit_models(implicit_models, holdout_yelp, training_matrix_yelp data_description_yelp)
scores_implicit_yelp.style.background_gradient()